In [2]:
! pip install opencv-python numpy tqdm matplotlib

In [3]:
import os
import cv2
import numpy as np
from tqdm import tqdm

In [4]:
REBUILD_DATA = False

class DogsVSCats():
    IMG_SIZE = 50
    CATS = "PetImages/Cat"
    DOGS = "PetImages/Dog"
    TESTING = "PetImages/Testing"
    LABELS = {CATS: 0, DOGS: 1}
    training_data = []

    catcount = 0
    dogcount = 0

    def make_training_data(self):
        for label in self.LABELS:
            print(label)
            for f in tqdm(os.listdir(label)):
                #print(f)
                if "jpg" in f:
                    try:
                        path = os.path.join(label, f)
                        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
                        img = cv2.resize(img, (self.IMG_SIZE, self.IMG_SIZE))
                        self.training_data.append([np.array(img), np.eye(2)[self.LABELS[label]]])
                        
                        if label == self.CATS:
                            self.catcount += 1
                        elif label == self.DOGS:
                            self.dogcount += 1
                    except Exception as e:
                        print(label, f, str(e))
                        pass
        np.random.shuffle(self.training_data)
        np.save("training_data.npy", self.training_data)
        print('Cats:', dogsvcats.catcount, self.catcount)
        print('Dogs:', dogsvcats.dogcount, self.dogcount)
        
if REBUILD_DATA:
    dogsvcats = DogsVSCats()
    dogsvcats.make_training_data()


In [5]:
training_data = np.load("training_data.npy", allow_pickle=True)
print(len(training_data))

24946


In [6]:
import torch

X = torch.Tensor([i[0] for i in training_data]).view(-1, 50, 50)
X = X/255.0
y = torch.Tensor([i[1] for i in training_data])

In [7]:
import matplotlib.pyplot as plt

plt.imshow(X[0], cmap="gray")
plt.show()

In [8]:
print(y[0])

tensor([1., 0.])


In [10]:
X.shape

torch.Size([24946, 50, 50])

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [29]:
class Net(nn.Module):
    def __init__(self):
        print("__init__()")
        super().__init__() 
        self.conv1 = nn.Conv2d(1, 32, 5)
        self.conv2 = nn.Conv2d(32, 64, 5)
        self.conv3 = nn.Conv2d(64, 128, 5)
        print(self.conv1, self.conv2, self.conv3)
        
        x = torch.randn(50, 50).view(-1, 1, 50, 50)
        self._to_linear = None
        self.convs(x)
        
        self.fc1 = nn.Linear(self._to_linear, 512)
        self.fc2 = nn.Linear(512, 2)
        
    def convs(self, x):
        print("convs()")
        #print("x0:", x[0].shape) # x0: torch.Size([1, 50, 50])
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        #print("x1:", x[0].shape)  # x1: torch.Size([32, 23, 23])
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        #print("x2:", x[0].shape) # x2: torch.Size([64, 9, 9])
        x = F.max_pool2d(F.relu(self.conv3(x)), (2, 2))
        #print("x3:", x[0].shape) # x3: torch.Size([128, 2, 2])
        
        if self._to_linear is None:
            print( x[0].shape[0], x[0].shape[1], x[0].shape[2])
            self._to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2]
        return x
            
    def forward(self, x):
        print("forward()")
        x = self.convs(x)
        x = x.view(-1, self._to_linear)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim=1)
    
net = Net()
print(net)

__init__()
Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1)) Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1)) Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1))
convs()
128 2 2
Net(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=512, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=2, bias=True)
)


In [30]:
import torch.optim as optim

loss_function = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [31]:
VAL_PCT = 0.1
val_size = int(len(X)*VAL_PCT)
print(val_size)

2494


In [32]:
train_X = X[:-val_size]
train_y = y[:-val_size]

test_X = X[-val_size:]
test_y = y[-val_size:]
print(len(train_X), len(test_X))

22452 2494


In [33]:
BATCH_SIZE = 100
EPOCHS = 1

for epoch in range(EPOCHS):
    for i in tqdm(range(0, len(train_X), BATCH_SIZE)): 
        batch_X = train_X[i:i+BATCH_SIZE].view(-1, 1, 50, 50)
        batch_y = train_y[i:i+BATCH_SIZE]
        
        net.zero_grad()
        
        outputs = net(batch_X)
        loss = loss_function(outputs, batch_y)
        loss.backward()
        optimizer.step()
        
    print(f"Epoch: {epoch}. Loss: {loss}")

  0%|          | 0/225 [00:00<?, ?it/s]

forward()
convs()


  0%|          | 1/225 [00:00<03:34,  1.04it/s]

forward()
convs()


  1%|          | 2/225 [00:01<03:23,  1.10it/s]

...

forward()
convs()


100%|█████████▉| 224/225 [03:02<00:00,  1.26it/s]

forward()
convs()


100%|██████████| 225/225 [03:03<00:00,  1.47it/s]

Epoch: 0. Loss: 0.2542436122894287


In [34]:
correct = 0 
total = 0

with torch.no_grad():
    for i in tqdm(range(len(test_X))):
        real_class = torch.argmax(test_y[i])
        net_out = net(test_X[i].view(-1, 1, 50, 50))[0]
        predicted_class = torch.argmax(net_out)
        
        if predicted_class == real_class:
            correct += 1
        total += 1

print("Accuracy:", round(correct/total, 3))

  0%|          | 11/2494 [00:00<00:23, 107.54it/s]

forward()
convs()
forward()
convs()
forward()
convs()
forward()
convs()
forward()
convs()
forward()
convs()
forward()
convs()
forward()
convs()
forward()
convs()
forward()
convs()
forward()
convs()
forward()
convs()
forward()
convs()
forward()
convs()
forward()
convs()
forward()
convs()


  1%|          | 29/2494 [00:00<00:20, 121.19it/s]

forward()
convs()
forward()
convs()
forward()
convs()
forward()
convs()
forward()
convs()
forward()
convs()
forward()
convs()
forward()
convs()
forward()
convs()
forward()
convs()
forward()
convs()
forward()
convs()
forward()
convs()
forward()
convs()


  2%|▏         | 47/2494 [00:00<00:18, 133.81it/s]

...

forward()
convs()
forward()
convs()
forward()
convs()
forward()
convs()
forward()
convs()
forward()
convs()
forward()
convs()
forward()
convs()
forward()
convs()
forward()
convs()
forward()
convs()
forward()
convs()
forward()
convs()
forward()
convs()
forward()
convs()
forward()
convs()
forward()
convs()
forward()
convs()
forward()
convs()
forward()
convs()


 99%|█████████▉| 2479/2494 [00:11<00:00, 185.63it/s]

forward()
convs()
forward()
convs()
forward()
convs()
forward()
convs()
forward()
convs()
forward()
convs()
forward()
convs()
forward()
convs()
forward()
convs()
forward()
convs()
forward()
convs()
forward()
convs()
forward()
convs()
forward()
convs()
forward()
convs()
forward()
convs()
forward()
convs()
forward()
convs()


100%|██████████| 2494/2494 [00:11<00:00, 209.13it/s]

forward()
convs()
forward()
convs()
forward()
convs()
forward()
convs()
forward()
convs()
forward()
convs()
forward()
convs()
forward()
convs()
forward()
convs()
Accuracy: 0.649
